# Extended tracing

Let's take this weird `nn.Module`, which is totally useless besides demonstrating how `torchbend`'s tracer extends the original `torch.fx`'s one and experiment tracing and bending.

In [ ]:
import torch, torch.nn as nn
import torchbend as tb

class Doug(nn.Module):
    def __init__(self):
        self.conv_module = nn.Conv1d(1, 16)
        self.batch_norm = nn.BatchNorm1d(16)
        
    def forward(self, x: torch.Tensor):
        if x.shape[1] > 1: 
            return torch.cat([self.forward(x[:, [i]]) for i in range(x.shape[1])], -1)
        else:
            outs = []
            for i in range(x.shape[0]):
                out_tmp = self.conv_module(x[[i]])
                out_tmp = self.batch_norm(out_tmp)
                outs.append(out_tmp)
            return sum(outs)